In [1]:
# Ensure GPU is available
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import files
uploaded = files.upload()


Saving connect4_cnn_model (1).h5 to connect4_cnn_model (1).h5
Saving best_model_combined_PlusOnly.h5 to best_model_combined_PlusOnly.h5
Saving connect4_cnn_2channel.h5 to connect4_cnn_2channel.h5
Saving CNN.h5 to CNN.h5
Saving cnn_model_feb10_merged_data_80epoch (1).h5 to cnn_model_feb10_merged_data_80epoch (1).h5


In [4]:
from tensorflow.keras.models import load_model

model_paths = {
    'Model_1': 'connect4_cnn_model (1).h5',
    'Model_2': 'best_model_combined_PlusOnly.h5',
    'Model_3': 'connect4_cnn_2channel.h5',
    'Model_4': 'CNN.h5',
    'Model_5': 'cnn_model_feb10_merged_data_80epoch (1).h5'
}

models = {name: load_model(path) for name, path in model_paths.items()}


In [5]:
ROWS, COLS = 6, 7

def create_board():
    return np.zeros((ROWS, COLS), dtype=int)

def get_valid_moves(board):
    return [c for c in range(COLS) if board[0][c] == 0]

def drop_piece(board, row, col, player):
    board[row][col] = player

def get_next_open_row(board, col):
    for r in reversed(range(ROWS)):
        if board[r][col] == 0:
            return r

def winning_move(board, player):
    for c in range(COLS - 3):
        for r in range(ROWS):
            if all(board[r][c+i] == player for i in range(4)):
                return True
    for c in range(COLS):
        for r in range(ROWS - 3):
            if all(board[r+i][c] == player for i in range(4)):
                return True
    for c in range(COLS - 3):
        for r in range(ROWS - 3):
            if all(board[r+i][c+i] == player for i in range(4)):
                return True
    for c in range(COLS - 3):
        for r in range(3, ROWS):
            if all(board[r-i][c+i] == player for i in range(4)):
                return True
    return False


In [6]:
def get_model_move(model, board, player):
    # Channel 0 = current player's pieces, Channel 1 = opponent's
    board_input = np.zeros((2, 6, 7))
    board_input[0] = (board == player)
    board_input[1] = (board == 3 - player)

    # Reshape to match model input shape: (1, 6, 7, 2)
    input_tensor = board_input.transpose(1, 2, 0).reshape(1, 6, 7, 2)

    preds = model.predict(input_tensor, verbose=0)[0]

    # Mask out illegal moves
    valid_moves = get_valid_moves(board)
    sorted_moves = np.argsort(preds)[::-1]  # Highest prob to lowest

    for move in sorted_moves:
        if move in valid_moves:
            return move

    # Fallback
    return random.choice(valid_moves)


In [7]:
for name, model in models.items():
    print(f"{name} input shape: {model.input_shape}")


Model_1 input shape: (None, 6, 7, 2)
Model_2 input shape: (None, 6, 7, 2)
Model_3 input shape: (None, 6, 7, 2)
Model_4 input shape: (None, 6, 7, 2)
Model_5 input shape: (None, 6, 7, 2)


Simulation

In [8]:
def play_game(model1, model2, first_player):
    board = create_board()
    player_turn = 1 if first_player == 1 else 2
    model_map = {1: model1, 2: model2}

    while True:
        model = model_map[player_turn]
        move = get_model_move(model, board, player_turn)
        row = get_next_open_row(board, move)
        drop_piece(board, row, move, player_turn)

        if winning_move(board, player_turn):
            return player_turn  # model1 = 1, model2 = 2

        if len(get_valid_moves(board)) == 0:
            return 0  # Draw

        player_turn = 3 - player_turn  # Switch player


In [26]:
from collections import defaultdict
import pandas as pd
import random
import numpy as np

# 🔧 Set how many games per unique matchup here
num_games = 100

model_names = list(models.keys())
win_matrix = pd.DataFrame(0, index=model_names, columns=model_names)
first_move_stats = defaultdict(lambda: {'games': 0, 'wins': 0})
total_game_counts = defaultdict(int)
tie_counts = defaultdict(int)

game_counter = 1
total_games = num_games * (len(model_names) * (len(model_names) - 1)) // 2

for i in range(len(model_names)):
    for j in range(i + 1, len(model_names)):  # Unique model pairs
        m1_name, m2_name = model_names[i], model_names[j]
        model1, model2 = models[m1_name], models[m2_name]

        for _ in range(num_games):
            # Randomly pick who plays first
            first_player = random.choice([1, 2])

            if first_player == 1:
                first_model_name = m1_name
                second_model_name = m2_name
                winner = play_game(model1, model2, first_player=1)
            else:
                first_model_name = m2_name
                second_model_name = m1_name
                winner = play_game(model1, model2, first_player=2)

            # Record first move count
            first_move_stats[first_model_name]['games'] += 1

            # Record result
            if winner == 1:
                win_matrix.at[m1_name, m2_name] += 1
                if first_player == 1:
                    first_move_stats[m1_name]['wins'] += 1
            elif winner == 2:
                win_matrix.at[m2_name, m1_name] += 1
                if first_player == 2:
                    first_move_stats[m2_name]['wins'] += 1
            else:
                tie_counts[m1_name] += 1
                tie_counts[m2_name] += 1

            # Record total games
            total_game_counts[m1_name] += 1
            total_game_counts[m2_name] += 1

            print(f"✅ Game {game_counter}/{total_games} complete")
            game_counter += 1


✅ Game 1/1000 complete
✅ Game 2/1000 complete
✅ Game 3/1000 complete
✅ Game 4/1000 complete
✅ Game 5/1000 complete
✅ Game 6/1000 complete
✅ Game 7/1000 complete
✅ Game 8/1000 complete
✅ Game 9/1000 complete
✅ Game 10/1000 complete
✅ Game 11/1000 complete
✅ Game 12/1000 complete
✅ Game 13/1000 complete
✅ Game 14/1000 complete
✅ Game 15/1000 complete
✅ Game 16/1000 complete
✅ Game 17/1000 complete
✅ Game 18/1000 complete
✅ Game 19/1000 complete
✅ Game 20/1000 complete
✅ Game 21/1000 complete
✅ Game 22/1000 complete
✅ Game 23/1000 complete
✅ Game 24/1000 complete
✅ Game 25/1000 complete
✅ Game 26/1000 complete
✅ Game 27/1000 complete
✅ Game 28/1000 complete
✅ Game 29/1000 complete
✅ Game 30/1000 complete
✅ Game 31/1000 complete
✅ Game 32/1000 complete
✅ Game 33/1000 complete
✅ Game 34/1000 complete
✅ Game 35/1000 complete
✅ Game 36/1000 complete
✅ Game 37/1000 complete
✅ Game 38/1000 complete
✅ Game 39/1000 complete
✅ Game 40/1000 complete
✅ Game 41/1000 complete
✅ Game 42/1000 complete
✅

In [19]:
print("🔢 Win Matrix (Each cell = wins vs that opponent):")
display(win_matrix)


🔢 Win Matrix (Each cell = wins vs that opponent):


,Model_1,Model_2,Model_3,Model_4,Model_5
Model_1,0,2,5,2,2
Model_2,3,0,3,3,3
Model_3,0,0,0,0,0
Model_4,0,0,4,0,0
Model_5,0,0,1,2,0


In [18]:
first_move_df = pd.DataFrame([
    {'Model': name,
     'Games as First Player': stats['games'],
     'Wins as First Player': stats['wins'],
     'Win % as First Player': round(100 * stats['wins'] / stats['games'], 1) if stats['games'] > 0 else 0}
    for name, stats in first_move_stats.items()
])

first_move_df = first_move_df.sort_values(by='Win % as First Player', ascending=False)
print("🎯 First Move Performance:")
display(first_move_df)


🎯 First Move Performance:


,Model,Games as First Player,Wins as First Player,Win % as First Player
0,Model_2,12,12,100.0
1,Model_1,11,11,100.0
3,Model_5,8,3,37.5
2,Model_4,12,4,33.3
4,Model_3,7,0,0.0


In [27]:
summary_data = []

for model in model_names:
    total_played = total_game_counts[model]
    total_won = sum(win_matrix.loc[model])
    ties = tie_counts[model]
    losses = total_played - total_won - ties
    first_played = first_move_stats[model]['games']
    first_wins = first_move_stats[model]['wins']

    summary_data.append({
        'Model': model,
        'Games Played': total_played,
        'Games Won': total_won,
        'Games Lost': losses,
        'Ties': ties,
        'Games Played First': first_played,
        'Games Won When First': first_wins
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values(by='Games Won', ascending=False)

display(summary_df)


,Model,Games Played,Games Won,Games Lost,Ties,Games Played First,Games Won When First
0,Model_1,400,351,49,0,198,198
1,Model_2,400,308,51,41,197,197
4,Model_5,400,200,159,41,185,90
3,Model_4,400,100,300,0,202,39
2,Model_3,400,0,400,0,218,0
